# Graph Convolutional Prediction of Protein Interactions in Yeast

*(This demo is a part of [Deep Learning for Network Biology](http://snap.stanford.edu/deepnetbio-ismb/) tutorial.)*

In this example, we demonstrate the utility of deep learning methods for an important prediction problem on biological graphs. In particular, we consider the problem of predicting [protein-protein interactions](https://en.wikipedia.org/wiki/Protein%E2%80%93protein_interaction) (PPIs). 

Protein-protein interactions (PPIs) are essential to almost every process in a cell. Understanding PPIs is crucial for understanding cell physiology in normal and disease states. Furthermore, knowledge of PPIs can be used:
* for drug development, since drugs can affect PPIs,
* to assign roles (i.e., protein functions) to uncharacterized proteins,
* to characterize the relationships between proteins that form multi-molecular complexes, such as the proteasome.

We represent the totality of PPIs that happen in a cell, an organism or a specific biological context with a [protein-protein interaction network](https://www.nature.com/subjects/protein-protein-interaction-networks). These networks are mathematical representations of all physical contacts between proteins in the cell.

The development of large-scale PPI screening techniques, especially [high-throughput affinity purification combined with mass-spectrometry](https://doi.org/10.1016/j.tibtech.2016.02.014) and the [yeast two-hybrid assay](https://en.wikipedia.org/wiki/Two-hybrid_screening), has caused an explosion in the amount of PPI data and the construction of ever more complex and complete interaction networks. For example, the figure below is a graphical representation of three different types of protein-protein interaction networks in [yeast *S. cerevisiae*](https://en.wikipedia.org/wiki/Saccharomyces_cerevisiae). The structure of the binary interaction network is obviously different from the structure of the co-complex interaction network. The network structure of the literature-curated dataset resembles that of the co-complex dataset, even though the literature-curated datasets are reported to contain mostly binary interactions.

However, current knowledge of protein-protein interaction networks is both [*incomplete* and *noisy*](https://doi.org/10.1016/j.cell.2014.10.050), as PPI screening techniques are limited in how many true interactions they can detect. Furthermore, PPI screening techniques often have high false positive and negative rates. These limitations present a great opportunity for computational methods to predict protein-protein interactions.  


<center>
<img src="yeast_interactome.png" alt="Yeast protein-protein interaction networks" style="width: 800px"/>
</center>

Image taken from: http://interactome.dfci.harvard.edu/S_cerevisiae

In this demo, we take the yeast protein-protein interaction network ([download the preprocessed network](http://snap.stanford.edu/deepnetbio-ismb/ipynb/yeast.edgelist)) and use the network to build a model for predicting new protein-protein interactions. We formulate this prediction task as **a link prediction problem on unweighted and undirected networks** and use a graph convolutional neural network to solve the task.

Note that link prediction is a standard prediction task in network biology, and thus this example can be easily extended and adapted to many other biomedical applications. 

## Graph Convolutional Neural Network (GCN)

In what follows, we give a complete [Tensorflow](https://www.tensorflow.org/) implementation of a two-layer graph convolutional neural network (GCN) for link prediction. We closely follow the GCN formulation as presented in [Kipf et al., ICLR 2017](https://arxiv.org/pdf/1609.02907.pdf). 

In [1]:
from __future__ import division
from __future__ import print_function

import time

import numpy as np
import scipy.sparse as sp
import gc

import networkx as nx
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

/Users/fujitsuka/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 20, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 32, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('hidden2', 16, 'Number of units in hidden layer 2.')
flags.DEFINE_integer('hidden3', 16, 'Number of units in hidden layer 3.')
flags.DEFINE_integer('hidden4', 16, 'Number of units in hidden layer 4.')
flags.DEFINE_float('dropout', 0.1, 'Dropout rate (1 - keep probability).')

## Various Utility Functions

In [6]:
def load_data(datapath):
    """
    edgelistデータを読み込み，　sp.sparse型の隣接行列を返す
    """
    g = nx.read_edgelist(datapath)
    adj = nx.adjacency_matrix(g, nodelist=sorted(g.nodes()))
    return adj.T


def weight_variable_glorot(input_dim, output_dim, name=""):
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = tf.random_uniform(
        [input_dim, output_dim], minval=-init_range,
        maxval=init_range, dtype=tf.float32)
    return tf.Variable(initial, name=name)


def dropout_sparse(x, keep_prob, num_nonzero_elems):
    noise_shape = [num_nonzero_elems]
    random_tensor = keep_prob
    random_tensor += tf.random_uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
    pre_out = tf.sparse_retain(x, dropout_mask)
    return pre_out * (1. / keep_prob)


def sparse_to_tuple(sparse_mx):
    """
    sp.sparse行列を，　値を持つcoords，　値の中身, 本来の行列の形のtupleに変換
    """
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape


def preprocess_graph(adj):
    """
    sp.sparse型の隣接行列を，　次数行列で正規化して，　coords, values, shape のtupleに変換
    """
    adj = sp.coo_matrix(adj)
    adj_ = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
    return sparse_to_tuple(adj_normalized)


def construct_feed_dict(adj_normalized, adj, features, placeholders):
    feed_dict = dict()
    feed_dict.update({placeholders['features']: features})
    feed_dict.update({placeholders['adj']: adj_normalized})
    feed_dict.update({placeholders['adj_orig']: adj})
    return feed_dict


def mask_test_edges(adj):
    # Function to build test set with 2% positive links (全体の1/50のリンク)
    # ここでは，　2%の正解に合わせて，　同じ数の不正解データも作っておく （精度を求める際に使用）
    
    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()

    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    num_test = int(np.floor(edges.shape[0] / 50.))
    num_val = int(np.floor(edges.shape[0] / 50.))
    
    # エッジのidをシャッフルして，　訓練用, 検証用と，テスト用のエッジに分ける
    all_edge_idx = [i for i in range(edges.shape[0])]
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)  # 配列edgesから，　テストと検証用のエッジを除く

    def ismember(a, b):
        rows_close = np.all((a - b[:, None]) == 0, axis=-1)
        return np.any(rows_close)

    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        n_rnd = len(test_edges) - len(test_edges_false) 
        rnd = np.random.randint(0, adj.shape[0], size=2 * n_rnd)  # 2*n_rnd 個のリスト（各要素は，　ランダムに選ばれたノードの番号）
        idxs_i = rnd[:n_rnd]  # 　半分を i と j それぞれに分ける                                      
        idxs_j = rnd[n_rnd:]
        for i in range(n_rnd):
            idx_i = idxs_i[i]
            idx_j = idxs_j[i]
            if idx_i == idx_j:
                continue
            if ismember([idx_i, idx_j], edges_all):
                continue
            if test_edges_false:  # リストが空でないなら，　True
                if ismember([idx_j, idx_i], np.array(test_edges_false)):
                    continue
                if ismember([idx_i, idx_j], np.array(test_edges_false)):
                    continue
            test_edges_false.append([idx_i, idx_j])  # 上記の条件を満たさない場合だけ，　test_edges_falseに加える

    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        n_rnd = len(val_edges) - len(val_edges_false)
        rnd = np.random.randint(0, adj.shape[0], size=2 * n_rnd)
        idxs_i = rnd[:n_rnd]                                        
        idxs_j = rnd[n_rnd:]
        for i in range(n_rnd):
            idx_i = idxs_i[i]
            idx_j = idxs_j[i]
            if idx_i == idx_j:
                continue
            if ismember([idx_i, idx_j], train_edges):
                continue
            if ismember([idx_j, idx_i], train_edges):
                continue
            if ismember([idx_i, idx_j], val_edges):
                continue
            if ismember([idx_j, idx_i], val_edges):
                continue
            if val_edges_false:
                if ismember([idx_j, idx_i], np.array(val_edges_false)):
                    continue
                if ismember([idx_i, idx_j], np.array(val_edges_false)):
                    continue
            val_edges_false.append([idx_i, idx_j])

    # Re-build adj matrix
    data = np.ones(train_edges.shape[0])
    adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
    adj_train = adj_train + adj_train.T

    return adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false


def get_roc_score(edges_pos, edges_neg):
    """
    評価用の，　正解エッジのリストと，　不正解エッジのリストを渡す
    """
    feed_dict.update({placeholders['dropout']: 0})  # 評価なので，　dropoutは0に設定　　
    emb = sess.run(model.embeddings, feed_dict=feed_dict)

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Predict on test set of edges
    adj_rec = np.dot(emb, emb.T)  # embは(N, F)行列ゆえ，　左のように積を取ると，　(N, N)で(i, j)成分はノード i と j の類似度を表す
    
    # positive edge と negative edge の2つで別々に評価
    preds = []
    pos = []
    for e in edges_pos:
        preds.append(sigmoid(adj_rec[e[0], e[1]]))
        pos.append(adj_orig[e[0], e[1]])

    preds_neg = []
    neg = []
    for e in edges_neg:
        preds_neg.append(sigmoid(adj_rec[e[0], e[1]]))
        neg.append(adj_orig[e[0], e[1]])

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)

    return roc_score, ap_score

## Define Convolutional Layers for our GCN Model

In [7]:
class GraphConvolution():
    """Basic graph convolution layer for undirected graph without edge labels."""
    def __init__(self, input_dim, output_dim, adj, name, dropout=0., act=tf.nn.relu):
        self.name = name
        self.vars = {}
        self.issparse = False
        with tf.variable_scope(self.name + '_vars'):
            self.vars['weights'] = weight_variable_glorot(input_dim, output_dim, name='weights')
        self.dropout = dropout
        self.adj = adj
        self.act = act

    def __call__(self, inputs):
        with tf.name_scope(self.name):        
            x = inputs
            x = tf.nn.dropout(x, 1-self.dropout)
            x = tf.matmul(x, self.vars['weights'])
            x = tf.sparse_tensor_dense_matmul(self.adj, x)
            outputs = self.act(x)
        return outputs


class GraphConvolutionSparse():
    """Graph convolution layer for sparse inputs."""
    def __init__(self, input_dim, output_dim, adj, features_nonzero, name, dropout=0., act=tf.nn.relu):
        self.name = name
        self.vars = {}
        self.issparse = False
        with tf.variable_scope(self.name + '_vars'):
            self.vars['weights'] = weight_variable_glorot(input_dim, output_dim, name='weights')
        self.dropout = dropout
        self.adj = adj
        self.act = act
        self.issparse = True
        self.features_nonzero = features_nonzero

    def __call__(self, inputs):
        with tf.name_scope(self.name):
            x = inputs
            x = dropout_sparse(x, 1-self.dropout, self.features_nonzero)
            x = tf.sparse_tensor_dense_matmul(x, self.vars['weights'])
            x = tf.sparse_tensor_dense_matmul(self.adj, x)
            outputs = self.act(x)
        return outputs
    
    
class InnerProductDecoder():
    """Decoder model layer for link prediction."""
    def __init__(self, input_dim, name, dropout=0., act=tf.nn.sigmoid):
        self.name = name
        self.issparse = False
        self.dropout = dropout
        self.act = act

    def __call__(self, inputs):
        with tf.name_scope(self.name):
            inputs = tf.nn.dropout(inputs, 1-self.dropout)
            x = tf.transpose(inputs)
            x = tf.matmul(inputs, x)  # 特徴ベクトルの内積，　(N, N)で各成分がノードとノードの類似度を表す
            x = tf.reshape(x, [-1])
            outputs = self.act(x)
        return outputs

In [8]:
class GraphConvolution_Attention():
    """
    attentionつき
    """
    def __init__(self, input_dim, output_dim, adj, name, dropout=0., act=tf.nn.relu):
        self.name = name
        self.vars = {}
        self.issparse = False
        self.dropout = dropout
        self.adj = adj
        self.act = act
        with tf.variable_scope(self.name + '_vars'):
            self.vars['weights'] = weight_variable_glorot(input_dim, output_dim, name='weights')
            self.vars['attention_self'] = weight_variable_glorot(output_dim, 1, name="attention_self")
            self.vars['attention_neigh'] = weight_variable_glorot(output_dim, 1, name="attention_neigh")


    def __call__(self, inputs):
        with tf.name_scope(self.name):
            x = inputs
            x = tf.nn.dropout(x, 1-self.dropout)
            x = tf.matmul(x, self.vars['weights'])

            # XWa = XWa_{self} + XWa_{neigh} と分ける
            att_self = tf.matmul(x, self.vars['attention_self'])  # (N,1)
            att_neigh = tf.matmul(x, self.vars['attention_neigh'])  # (N,1)
            att = tf.add(att_self, tf.transpose(att_neigh))  # (N,1) + (1,N)　→ (N,N)にbroadcastされるのを利用
            att = self.adj.__mul__(att)  # 隣接行列と要素積を取ったもの(__mul__でsparse型との要素積が可能, 結果はsparseとなる)
            att = tf.SparseTensor(indices=att.indices, values=tf.nn.leaky_relu(att.values, alpha=0.2), dense_shape=att.dense_shape)
            att = tf.sparse_softmax(att)
            
            x = tf.sparse_tensor_dense_matmul(att, x)
            #x = tf.matmul(att, x)
            outputs = self.act(x)
        return outputs

    
class GraphConvolutionSparse_Attention():
    """Graph convolution layer for sparse inputs."""
    def __init__(self, input_dim, output_dim, adj, features_nonzero, name, dropout=0., act=tf.nn.relu):
        self.name = name
        self.vars = {}
        self.issparse = False
        self.dropout = dropout
        self.adj = adj
        self.act = act
        self.issparse = True
        self.features_nonzero = features_nonzero
        with tf.variable_scope(self.name + '_vars'):
            self.vars['weights'] = weight_variable_glorot(input_dim, output_dim, name='weights')
            self.vars['attention_self'] = weight_variable_glorot(output_dim, 1, name="attention_self")
            self.vars['attention_neigh'] = weight_variable_glorot(output_dim, 1, name="attention_neigh")
    
    #def attention(self, X):
        # XWの計算
        #X_linear_tr = tf.matmul(X, self.vars['weights'])
        # XWa = XWa_{self} + XWa_{neigh} と分ける
        #att_self = tf.matmul(X_linear_tr, self.vars['attention_self'])  # (N,1)
        #att_neigh = tf.matmul(X_linear_tr, self.vars['attention_neigh'])  # (N,1)
        #att = tf.add(att_self, tf.transpose(att_neigh))  # (N,1) + (1,N)　→ (N,N)にbroadcastされるのを利用

        #del X_linear_tr
        #gc.collect()

        #return self.adj.__mul__(att)  # 隣接行列と要素積を取ったもの(__mul__でsparse型との要素積が可能)

    def __call__(self, inputs):
        with tf.name_scope(self.name):
            x = inputs
            x = dropout_sparse(x, 1-self.dropout, self.features_nonzero)
            x = tf.sparse_tensor_dense_matmul(x, self.vars['weights'])  # この結果は， denseとなる
            
            # XWa = XWa_{self} + XWa_{neigh} と分ける
            att_self = tf.matmul(x, self.vars['attention_self'])  # (N,1)
            att_neigh = tf.matmul(x, self.vars['attention_neigh'])  # (N,1)
            
            # SparseTensor型で，　attentionの重みを作成
            # 注意として，　placeholderは入るまで分からないため，　型とか値とかは，　この時点では，　？ なので，　例えば，　
            #  att = tf.SparseTensor(indices=self.adj.indices, values=[att_self[self.adj.indices[i][0]]+att_neigh[self.adj.indices[i][1]] for i in range(self.adj.indices[0].value)], dense_shape=self.adj.dense_shape)
            # 上記は，　self.adj.indices[0].valueがNoneゆえ，　ループが回らなく，　エラーとなる
            
            # (N,N)と(N,1)の要素積は，　(N,1)を(N,N)の列方向に各列に掛けていったものになる
            # (N,N)と(1,N)の要素積は，　(1,N)を(N,N)の行方向に各行に掛けていったものになる
            
            att_1 = self.adj.__mul__(tf.nn.leaky_relu(att_self, alpha=0.2))
            att_2 = self.adj.__mul__(tf.transpose(tf.nn.leaky_relu(att_neigh, alpha=0.2)))
            att = tf.sparse_add(att_1, att_2)
            
            del att_1,   att_2
            gc.collect()
            
            #att = tf.add(att_self, tf.transpose(att_neigh))  # (N,1) + (1,N)　→ (N,N)にbroadcastされるのを利用
            #att = self.adj.__mul__(att)  # 隣接行列と要素積を取ったもの(adjはsparseゆえ，　__mul__でsparse型との要素積， 結果はsparseとなる)
            #att = tf.SparseTensor(indices=att.indices, values=tf.nn.leaky_relu(att.values), dense_shape=att.dense_shape)
            
            att = tf.sparse_softmax(att)
            
            x = tf.sparse_tensor_dense_matmul(att, x)
            outputs = self.act(x)
        return outputs

## Specify the Architecture of our GCN Model

In [9]:
class GCNModel():
    def __init__(self, placeholders, num_features, features_nonzero, name):
        self.name = name
        self.inputs = placeholders['features']
        self.input_dim = num_features
        self.features_nonzero = features_nonzero
        self.adj = placeholders['adj']
        self.dropout = placeholders['dropout']
        self.vars = {}
        with tf.variable_scope(self.name):
            self.vars['attention_layer'] = tf.Variable([1,1,1], dtype=tf.float32, name="attention_layer")
            self.build()
        
    def build(self):
        self.hidden1 = GraphConvolutionSparse_Attention(
            name='gcn_sparse_layer',
            input_dim=self.input_dim,
            output_dim=FLAGS.hidden1,
            adj=self.adj,
            features_nonzero=self.features_nonzero,
            act=tf.nn.relu,
            dropout=self.dropout)(self.inputs)

        self.hidden2 = GraphConvolution_Attention(
            name='gcn_dense_layer_1',
            input_dim=FLAGS.hidden1,
            output_dim=FLAGS.hidden2,
            adj=self.adj,
            act=lambda x: x,
            dropout=self.dropout)(self.hidden1)
        
        self.hidden3 = GraphConvolution_Attention(
            name='gcn_dense_layer_2',
            input_dim=FLAGS.hidden2,
            output_dim=FLAGS.hidden3,
            adj=self.adj,
            act=lambda x: x,
            dropout=self.dropout)(self.hidden2)
        
        self.hidden4 = GraphConvolution_Attention(
            name='gcn_dense_layer_3',
            input_dim=FLAGS.hidden3,
            output_dim=FLAGS.hidden4,
            adj=self.adj,
            act=lambda x: x,
            dropout=self.dropout)(self.hidden3)
        
        weight = tf.nn.softmax(self.vars['attention_layer'] )
        self.embeddings = self.vars['attention_layer'][0]*self.hidden2 + self.vars['attention_layer'][1]*self.hidden3 + self.vars['attention_layer'][2]*self.hidden4

        self.reconstructions = InnerProductDecoder(
            name='gcn_decoder',
            input_dim=FLAGS.hidden4, 
            act=lambda x: x)(self.embeddings)

## Specify the GCN Optimizer

In [10]:
class Optimizer():
    def __init__(self, preds, labels, num_nodes, num_edges):
        
        # 重み付き交差エントロピー関数の重み
        # A value pos_weights > 1 decreases the false negative count, hence increasing the recall. 
        # Conversely setting pos_weights < 1 decreases the false positive count and increases the precision.
        pos_weight = float(num_nodes**2 - num_edges) / num_edges
        norm = num_nodes**2 / float((num_nodes**2 - num_edges) * 2)
        
        preds_sub = preds
        labels_sub = labels

        self.cost = norm * tf.reduce_mean(
            tf.nn.weighted_cross_entropy_with_logits(
                logits=preds_sub, targets=labels_sub, pos_weight=pos_weight))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)  # Adam Optimizer

        self.opt_op = self.optimizer.minimize(self.cost)
        self.grads_vars = self.optimizer.compute_gradients(self.cost)

## Train the GCN Model and Evaluate its Accuracy on a Test Set of Protein-Protein Interactions

Given a training set of protein-protein interactions in yeast *S. cerevisiae*, our goal is to take these interactions and train a GCN model that can predict new protein-protein interactions. That is, we would like to predict new edges in the yeast protein interaction network. 

In [11]:
adj = load_data('data/yeast.edgelist')
num_nodes = adj.shape[0]
num_edges = adj.sum()  # 今，　重みなしグラフを考えているので，　全て足すとエッジ数になる

# featureless
# 現状，　featuresがないので，　単位行列（one-hotベクトル）を入れる
features = sparse_to_tuple(sp.identity(num_nodes))
num_features = features[2][1]
features_nonzero = features[1].shape[0]  # [1]成分は，　ノンゼロvalues

# Store original adjacency matrix (without diagonal entries) for later
adj_orig = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
adj_orig.eliminate_zeros()

adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)
adj = adj_train

adj_norm = preprocess_graph(adj)

In [12]:
# Define placeholders
# 現状，　featureはないので，　sp.sparse表現でone-hotベクトルを入れているが，　ここは変更する（一般にsparse表現ではない）
placeholders = {
    'features': tf.sparse_placeholder(tf.float32),
    'adj': tf.sparse_placeholder(tf.float32),
    'adj_orig': tf.sparse_placeholder(tf.float32),
    'dropout': tf.placeholder_with_default(0., shape=())
}

# Create model
model = GCNModel(placeholders, num_features, features_nonzero, name='yeast_gcn')

# Create optimizer
# adj = adj_trainと上で代入しており，　訓練データに関してoptimizeするように設定
with tf.name_scope('optimizer'):
    opt = Optimizer(
        preds=model.reconstructions,
        labels=tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'], validate_indices=False), [-1]),
        num_nodes=num_nodes,
        num_edges=num_edges)

In [13]:
# Initialize session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

adj_label = adj_train + sp.eye(adj_train.shape[0])
adj_label = sparse_to_tuple(adj_label)

# Train model
for epoch in range(FLAGS.epochs):
    t = time.time()
    # Construct feed dictionary
    # placeholdersに代入する辞書作り
    feed_dict = construct_feed_dict(adj_norm, adj_label, features, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})
    # One update of parameter matrices
    _, avg_cost = sess.run([opt.opt_op, opt.cost], feed_dict=feed_dict)  # optimizeして，　コスト関数を出力
    # Performance on validation set
    roc_curr, ap_curr = get_roc_score(val_edges, val_edges_false)  # ただ各エポックで精度を見るためだけのもの

    print("Epoch:", '%04d' % (epoch + 1), 
          "train_loss=", "{:.5f}".format(avg_cost),
          "val_roc=", "{:.5f}".format(roc_curr),
          "val_ap=", "{:.5f}".format(ap_curr),
          "time=", "{:.5f}".format(time.time() - t))

print('Optimization Finished!')

roc_score, ap_score = get_roc_score(test_edges, test_edges_false)
print('Test ROC score: {:.5f}'.format(roc_score))
print('Test AP score: {:.5f}'.format(ap_score))

Epoch: 0001 train_loss= 0.71085 val_roc= 0.87384 val_ap= 0.86487 time= 22.08053
Epoch: 0002 train_loss= 0.68903 val_roc= 0.87541 val_ap= 0.86599 time= 21.28690
Epoch: 0003 train_loss= 0.63423 val_roc= 0.87597 val_ap= 0.86633 time= 22.02609
Epoch: 0004 train_loss= 0.61024 val_roc= 0.87627 val_ap= 0.86653 time= 21.93498
Epoch: 0005 train_loss= 0.63731 val_roc= 0.87638 val_ap= 0.86661 time= 22.64135
Epoch: 0006 train_loss= 0.61285 val_roc= 0.87645 val_ap= 0.86667 time= 21.95668
Epoch: 0007 train_loss= 0.60488 val_roc= 0.87658 val_ap= 0.86676 time= 22.39017
Epoch: 0008 train_loss= 0.61457 val_roc= 0.87675 val_ap= 0.86685 time= 21.57474
Epoch: 0009 train_loss= 0.61865 val_roc= 0.87692 val_ap= 0.86695 time= 22.31925
Epoch: 0010 train_loss= 0.61470 val_roc= 0.87708 val_ap= 0.86703 time= 22.68117
Epoch: 0011 train_loss= 0.60752 val_roc= 0.87720 val_ap= 0.86709 time= 22.03820
Epoch: 0012 train_loss= 0.60314 val_roc= 0.87731 val_ap= 0.86714 time= 22.30449
Epoch: 0013 train_loss= 0.60730 val_roc=